In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

10- A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias). Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. Con esta información escribir un programa en pySpark que permita:

a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales 

b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias).

In [3]:
datos_ventas_diarias = [
    (1, 'AR', '2018-01-01', 30, 10),
    (1, 'ES', '2018-01-01', 23, 13),
    (2, 'US', '2018-01-04', 45, 5),
    (2, 'MX', '2018-01-04', 20, 10),
    (2, 'US', '2018-01-06', 50, 15),
    (3, 'AR', '2018-01-06', 10, 2),
    (1, 'US', '2018-01-06', 14, 4),
    (3, 'ES', '2018-01-10', 34, 11),
    (4, 'ES', '2018-01-11', 42, 24),
    (4, 'US', '2018-01-11', 83, 34),
    (4, 'AR', '2018-01-11', 27, 20),
    (4, 'MX', '2018-01-11', 47, 18),
    (4, 'AR', '2018-01-20', 10, 0),
    (4, 'US', '2018-01-21', 34, 2),
    (4, 'ES', '2018-01-21', 25, 7)
]

datos_videojuegos = [
    (1, 'Zelda: Breath of the Wild', 9, 8),
    (2, 'Mario Kart', 9, 7),
    (3, 'Splatoon 2', 11, 8),
    (4, 'Monster Hunter Generations Ultimate', 13, 10)
]

In [5]:
ventas_rdd = sc.parallelize(datos_ventas_diarias)
videojuegos_rdd = sc.parallelize(datos_videojuegos)

a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias)

RDD2: (id_videojuego, titulo, rating_pegi, rating_esbr)

In [11]:
ventas_filtrado = ventas_rdd.map(lambda x: (x[0],x[4])).reduceByKey(lambda x,y: x+y).cache()

In [47]:
Mas_vendido = videojuegos_rdd.map(lambda x: (x[0], x[1])).join(ventas_filtrado)\
.map(lambda x: (x[0], x[1][0], x[1][1]))\
.reduce(lambda x,y: x if x[2]>y[2] else y)
Mas_vendido

(4, 'Monster Hunter Generations Ultimate', 105)

b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias).

In [44]:
ventas_rdd.filter(lambda x: x[0] == Mas_vendido[0]).map(lambda x: (x[1], (x[3],x[4])))\
.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
.map(lambda x: (x[0], (x[1][1]/x[1][0]))).reduce(lambda x,y: x if x[1] > y[1] else y)[0]

'AR'